# Preprocesado de Datos

Para este proyecto usar Polars, es una libreria similar a Pandas, pero optimizada para grandes volumenes de datos

Primero cargare cada uno de mis datasets y comprobare como unirlos.

In [2]:
import polars as pl
from pathlib import Path

In [3]:
BASE_DIR = Path.cwd()
DATA_DIR = (BASE_DIR / 'data').resolve()
RAW_DIR = (DATA_DIR / 'raw').resolve()
PROCESSED_DIR = (DATA_DIR / 'processed').resolve()

In [4]:
df_members = pl.scan_csv(RAW_DIR/'members_v3.csv')
df_train = pl.scan_csv(RAW_DIR/'train.csv')
df_transactions = pl.scan_csv(RAW_DIR/'transactions_v2.csv')
#df_user_logs = pl.scan_csv(RAW_DIR/'user_logs.csv')

print(df_members.head().collect())
print(df_train.head().collect())
print(df_transactions.head().collect())


shape: (5, 6)
┌─────────────────────────────────┬──────┬─────┬────────┬────────────────┬────────────────────────┐
│ msno                            ┆ city ┆ bd  ┆ gender ┆ registered_via ┆ registration_init_time │
│ ---                             ┆ ---  ┆ --- ┆ ---    ┆ ---            ┆ ---                    │
│ str                             ┆ i64  ┆ i64 ┆ str    ┆ i64            ┆ i64                    │
╞═════════════════════════════════╪══════╪═════╪════════╪════════════════╪════════════════════════╡
│ Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNL… ┆ 1    ┆ 0   ┆ null   ┆ 11             ┆ 20110911               │
│ +tJonkh+O1CA796Fm5X60UMOtB6POH… ┆ 1    ┆ 0   ┆ null   ┆ 7              ┆ 20110914               │
│ cV358ssn7a0f7jZOwGNWS07wCKVqxy… ┆ 1    ┆ 0   ┆ null   ┆ 11             ┆ 20110915               │
│ 9bzDeJP6sQodK73K5CBlJ6fgIQzPeL… ┆ 1    ┆ 0   ┆ null   ┆ 11             ┆ 20110915               │
│ WFLY3s7z4EZsieHCt63XrsdtfTEmJ+… ┆ 6    ┆ 32  ┆ female ┆ 9              ┆ 20110915   

Con esta observacion parcial de mis datasets, puedo ver que tengo una columna llamada *'msno'* que esta formada por strings y es la columna comun en los tres datasets y por donde tendre que hacer el join.

In [43]:
def lazy_info(df):
    print(f'Shape:\n', (df.select(pl.len()).collect().item(), df.collect_schema().len()))
    print('=' * 20)
    print(f'\nNombre de columna y su tipo de dato: \n', df.collect_schema())
    print('=' * 20)
    print(f'\nNumero de valores nulos:\n', df.null_count().collect())

In [44]:
lista_df = [df_members, df_train, df_transactions]

for df in lista_df:
    lazy_info(df)

Shape:
 (6769473, 6)

Nombre de columna y su tipo de dato: 
 Schema({'msno': String, 'city': Int64, 'bd': Int64, 'gender': String, 'registered_via': Int64, 'registration_init_time': Int64})

Numero de valores nulos:
 shape: (1, 6)
┌──────┬──────┬─────┬─────────┬────────────────┬────────────────────────┐
│ msno ┆ city ┆ bd  ┆ gender  ┆ registered_via ┆ registration_init_time │
│ ---  ┆ ---  ┆ --- ┆ ---     ┆ ---            ┆ ---                    │
│ u32  ┆ u32  ┆ u32 ┆ u32     ┆ u32            ┆ u32                    │
╞══════╪══════╪═════╪═════════╪════════════════╪════════════════════════╡
│ 0    ┆ 0    ┆ 0   ┆ 4429505 ┆ 0              ┆ 0                      │
└──────┴──────┴─────┴─────────┴────────────────┴────────────────────────┘
Shape:
 (992931, 2)

Nombre de columna y su tipo de dato: 
 Schema({'msno': String, 'is_churn': Int64})

Numero de valores nulos:
 shape: (1, 2)
┌──────┬──────────┐
│ msno ┆ is_churn │
│ ---  ┆ ---      │
│ u32  ┆ u32      │
╞══════╪══════════╡
│ 0   

Tras analizar, puedo ver que cada uno tiene dimensiones distintas, por lo que tendre que comprobar si la key de union se repite, y en caso de que lo haga, valorar como escojo los datos resultantes.

Ademas, todas mis columnas son numericas excepto 'gender', que es ademas la unica columna con valores NA.

In [63]:
# Contar registros totales vs registros únicos
total_records = df_members.select(pl.len()).collect().item()
unique_records = df_members.select(pl.col("msno").n_unique()).collect().item()

print(f"Total: {total_records:,}")
print(f"Únicos: {unique_records:,}")
print(f"¿Hay duplicados? {'SÍ' if total_records > unique_records else 'NO'}")

Total: 6,769,473
Únicos: 6,769,473
¿Hay duplicados? NO


En mi dataset members, no hay valores repetidos, esto es clave, ya que no puedo tener un usuario varias veces, el siguiente paso sera unir los datasets por la clave *'msno'*


In [ ]:
df_members = df_members.with_columns([
    pl.col("registration_init_time")
      .cast(pl.Utf8)
      .str.strptime(pl.Date, "%Y%m%d", strict=True)
      .alias("registration_date")
])

In [ ]:
df_transactions = df_transactions.with_columns([
    pl.col("transaction_date")
      .cast(pl.Utf8)
      .str.strptime(pl.Date, "%Y%m%d", strict=True)
    pl.col("registration_init_time")
      .cast(pl.Utf8)
      .str.strptime(pl.Date, "%Y%m%d", strict=True)
])

In [ ]:
last_transaction = df_transactions.group_by('msno').agg(pl.col("transaction_date").max().alias("last_transaction"))

df_cutoff = df_train.join(last_transaction, on='msno', how='inner')

df_cutoff = df_cutoff.sort('last_transaction')

df_p95 = df_cutoff.group_by('is_churn').agg(
    pl.col('last_transaction').quantile(0.95)
    .alias('p95_last_transaction')
    )

df_p95.head().collect()